In [1]:
from typing import Any, Generator, Protocol, List, Tuple
import pandas as pd
from alignscore import AlignScore
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/miaoran/.alignscore/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def AlignScoreEval(filename, model_size, col_name = None, update=True, batch_size=20, outputfile=None):
    df = pd.read_csv(filename, encoding='utf-8').fillna('')
    if not col_name:
        col_name = f'alignscore-{model_size}'
    if (not update) and (col_name in df): # store HHEM scores
        return
    if model_size == 'base':
        model = AlignScore(model='roberta-base', batch_size=batch_size, device='cuda:0', ckpt_path='/home/miaoran/AlignScore/AlignScore-base.ckpt', evaluation_mode='bin_sp')
    elif model_size == 'large':
        model = AlignScore(model='roberta-large', batch_size=batch_size, device='cuda:0', ckpt_path='/home/miaoran/AlignScore/AlignScore-large.ckpt', evaluation_mode='bin_sp')
    
    # scores = []
    # n = len(df['source'].tolist())
    # for i in range(0,n,batch_size):
    #     pred_label, raw_prob, _, _ = model.score(docs=df['source'].tolist()[i:i+batch_size], claims=df['summary'].tolist()[i:i+
    #                                                                                                                        batch_size])
    #     scores.extend(pred_label)
    scores = model.score(contexts=df['source'].tolist(), claims=df['summary'].tolist())

    if col_name in df:
        df[col_name] = scores
    else:
        df.insert(len(df.columns), col_name, scores)
    if not outputfile:
        outputfile = filename
    df.to_csv(outputfile, mode='w', index=False, header=True)
    print(f'AlignScore {model_size} Scores have been saved')


In [3]:
model_sizes = ['base', 'large']

for idx, model_size in enumerate(model_sizes):
    print(f"Run model {str(idx)}: {model_size} ......")
    filename = f"../assign/examples_to_annotate.csv"
    AlignScoreEval(filename, model_size=model_size, update=False)


Run model 0: base ......


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/miaoran/.alignscore/lib/python3.11/site-packages/lightning_fabric/utilities/cloud_io.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by

AlignScore base Scores have been saved
Run model 1: large ......


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/miaoran/.alignscore/lib/python3.11/site-packages/lightning_fabric/utilities/cloud_io.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted b

AlignScore large Scores have been saved
